In [75]:
# Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pyspark.sql import SparkSession

In [76]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("PySpark and Neural Network") \
    .getOrCreate()

In [77]:
# Load the dataset
dataset_path = "obesity_dataset.csv"
df = spark.read.csv(dataset_path, header = True, inferSchema = True)

## 1. Food between meals as predictor of obesity and overweight

In [78]:
# Select columns 'Food_Between_Meals' and 'Obese/Overweight'
data = df[['Food_Between_Meals', 'Obese/Overweight']]
data.head()

Row(Food_Between_Meals='Sometimes', Obese/Overweight='No')

In [79]:
# Convert to pandas dataframe
pandas_df = data.select("Food_Between_Meals", "Obese/Overweight").toPandas()
pandas_df.head()

,Food_Between_Meals,Obese/Overweight
0,Sometimes,No
1,Sometimes,No
2,Sometimes,No
3,Sometimes,Yes
4,Sometimes,Yes


In [6]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df['Food_Between_Meals'] = label_encoder.fit_transform(pandas_df['Food_Between_Meals'])
pandas_df['Obese/Overweight'] = label_encoder.fit_transform(pandas_df['Obese/Overweight'])

In [7]:
# Define features (X) and target (y)
X = pandas_df[['Food_Between_Meals']] 
y = pandas_df['Obese/Overweight']   

In [8]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [12]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8191 - loss: 0.6474 - val_accuracy: 0.8373 - val_loss: 0.5920
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.8131 - loss: 0.5786 - val_accuracy: 0.8373 - val_loss: 0.5363
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.8210 - loss: 0.5236 - val_accuracy: 0.8373 - val_loss: 0.5035
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - accuracy: 0.8231 - loss: 0.4975 - val_accuracy: 0.8373 - val_loss: 0.4891
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - accuracy: 0.8061 - loss: 0.5092 - val_accuracy: 0.8373 - val_loss: 0.4831
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.8060 - loss: 0.5276 - val_accuracy: 0.8373 - val_loss: 0.4796
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.8136 - loss: 0.4932 - val_accuracy: 0.8373 - val_loss: 0.4775
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.8162 - loss: 0.4998 - val_accura

In [13]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8436 - loss: 0.4369 
Neural Network Accuracy: 0.8368794322013855


## 2. Mode of transportation as predictor of obesity/overweight

In [14]:
# Select columns
data2 = df[['Mode_Transportation', 'Obese/Overweight']]
data2.head()

Row(Mode_Transportation='Public_Transportation', Obese/Overweight='No')

In [15]:
# Convert to pandas dataframe
pandas_df2 = data2.select("Mode_Transportation", "Obese/Overweight").toPandas()
pandas_df2.head()

,Mode_Transportation,Obese/Overweight
0,Public_Transportation,No
1,Public_Transportation,No
2,Public_Transportation,No
3,Walking,Yes
4,Public_Transportation,Yes


In [16]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df2['Mode_Transportation'] = label_encoder.fit_transform(pandas_df2['Mode_Transportation'])
pandas_df2['Obese/Overweight'] = label_encoder.fit_transform(pandas_df2['Obese/Overweight'])

In [17]:
# Define features (X) and target (y)
X = pandas_df2[['Mode_Transportation']]
y = pandas_df2['Obese/Overweight']   

In [18]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [22]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7144 - loss: 0.6488 - val_accuracy: 0.7544 - val_loss: 0.5926
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.7275 - loss: 0.6074 - val_accuracy: 0.7544 - val_loss: 0.5661
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.7176 - loss: 0.6042 - val_accuracy: 0.7544 - val_loss: 0.5593
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.7373 - loss: 0.5828 - val_accuracy: 0.7544 - val_loss: 0.5575
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.7192 - loss: 0.6044 - val_accuracy: 0.7544 - val_loss: 0.5574
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.7240 - loss: 0.5948 - val_accuracy: 0.7544 - val_loss: 0.5564
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.7219 - loss: 0.5943 - val_accuracy: 0.7544 - val_loss: 0.5557
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.7380 - loss: 0.5773 - val_accura

In [23]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7567 - loss: 0.5505 
Neural Network Accuracy: 0.73758864402771


## 3. Alcohol intake as predictor of obesity/overweight

In [24]:
# Select columns
data3 = df[['Alcohol_Intake', 'Obese/Overweight']]
data3.head()

Row(Alcohol_Intake='no', Obese/Overweight='No')

In [25]:
# Convert to pandas dataframe
pandas_df3 = data3.select("Alcohol_Intake", "Obese/Overweight").toPandas()
pandas_df3.head()

,Alcohol_Intake,Obese/Overweight
0,no,No
1,Sometimes,No
2,Frequently,No
3,Frequently,Yes
4,Sometimes,Yes


In [26]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df3['Alcohol_Intake'] = label_encoder.fit_transform(pandas_df3['Alcohol_Intake'])
pandas_df3['Obese/Overweight'] = label_encoder.fit_transform(pandas_df3['Obese/Overweight'])

In [27]:
# Define features (X) and target (y)
X = pandas_df3[['Alcohol_Intake']]
y = pandas_df3['Obese/Overweight']   

In [28]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [32]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3695 - loss: 0.7174 - val_accuracy: 0.7544 - val_loss: 0.6592
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - accuracy: 0.7292 - loss: 0.6538 - val_accuracy: 0.7544 - val_loss: 0.6204
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.7191 - loss: 0.6249 - val_accuracy: 0.7544 - val_loss: 0.5885
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.7212 - loss: 0.6027 - val_accuracy: 0.7544 - val_loss: 0.5684
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.7255 - loss: 0.5878 - val_accuracy: 0.7544 - val_loss: 0.5590
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.7274 - loss: 0.5837 - val_accuracy: 0.7544 - val_loss: 0.5567
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - accuracy: 0.7345 - loss: 0.5746 - val_accuracy: 0.7544 - val_loss: 0.5553
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - accuracy: 0.7264 - loss: 0.5874 - val_accura

In [33]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.5556 
Neural Network Accuracy: 0.7352246046066284


## 4. Smoking as predictor of obesity/overweight

In [34]:
# Select columns
data4 = df[['SMOKE', 'Obese/Overweight']]
data4.head()

Row(SMOKE='no', Obese/Overweight='No')

In [35]:
# Convert to pandas dataframe
pandas_df4 = data4.select("SMOKE", "Obese/Overweight").toPandas()
pandas_df4.head()

,SMOKE,Obese/Overweight
0,no,No
1,yes,No
2,no,No
3,no,Yes
4,no,Yes


In [36]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df4['SMOKE'] = label_encoder.fit_transform(pandas_df4['SMOKE'])
pandas_df4['Obese/Overweight'] = label_encoder.fit_transform(pandas_df4['Obese/Overweight'])

In [37]:
# Define features (X) and target (y)
X = pandas_df4[['SMOKE']]
y = pandas_df4['Obese/Overweight']   

In [38]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [39]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [40]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [42]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4957 - loss: 0.7209 - val_accuracy: 0.7515 - val_loss: 0.6754
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 748us/step - accuracy: 0.7269 - loss: 0.6757 - val_accuracy: 0.7515 - val_loss: 0.6243
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - accuracy: 0.7305 - loss: 0.6228 - val_accuracy: 0.7515 - val_loss: 0.5877
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - accuracy: 0.7293 - loss: 0.5968 - val_accuracy: 0.7544 - val_loss: 0.5680
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.7205 - loss: 0.5943 - val_accuracy: 0.7544 - val_loss: 0.5609
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.7395 - loss: 0.5759 - val_accuracy: 0.7544 - val_loss: 0.5594
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.7411 - loss: 0.5719 - val_accuracy: 0.7544 - val_loss: 0.5596
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.7154 - loss: 0.5980 - val_accura

In [43]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.5619
Neural Network Accuracy: 0.7352246046066284


## 5. Family history as predictor of obesity/overweight

In [44]:
# Select columns
data5 = df[['family_history_with_overweight', 'Obese/Overweight']]
data5.head()

Row(family_history_with_overweight='yes', Obese/Overweight='No')

In [45]:
# Convert to pandas dataframe
pandas_df5 = data5.select("family_history_with_overweight", "Obese/Overweight").toPandas()
pandas_df5.head()

,family_history_with_overweight,Obese/Overweight
0,yes,No
1,yes,No
2,yes,No
3,no,Yes
4,no,Yes


In [46]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df5['family_history_with_overweight'] = label_encoder.fit_transform(pandas_df5['family_history_with_overweight'])
pandas_df5['Obese/Overweight'] = label_encoder.fit_transform(pandas_df5['Obese/Overweight'])

In [47]:
# Define features (X) and target (y)
X = pandas_df5[['family_history_with_overweight']]
y = pandas_df5['Obese/Overweight']   

In [48]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [49]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [50]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [52]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7570 - loss: 0.6369 - val_accuracy: 0.8047 - val_loss: 0.5731
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 0.8133 - loss: 0.5554 - val_accuracy: 0.8047 - val_loss: 0.5148
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.8119 - loss: 0.5089 - val_accuracy: 0.8047 - val_loss: 0.4903
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.8285 - loss: 0.4631 - val_accuracy: 0.8047 - val_loss: 0.4869
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.7922 - loss: 0.5046 - val_accuracy: 0.8047 - val_loss: 0.4867
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.8197 - loss: 0.4630 - val_accuracy: 0.8047 - val_loss: 0.4872
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.7984 - loss: 0.4962 - val_accuracy: 0.8047 - val_loss: 0.4869
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - accuracy: 0.8150 - loss: 0.4703 - val_accura

In [53]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - accuracy: 0.8514 - loss: 0.4192
Neural Network Accuracy: 0.8368794322013855


## 6. High-caloric food as predictor of obesity/overweight

In [54]:
# Select columns
data6 = df[['High_Caloric_Food', 'Obese/Overweight']]
data6.head()

Row(High_Caloric_Food='no', Obese/Overweight='No')

In [55]:
# Convert to pandas dataframe
pandas_df6 = data6.select("High_Caloric_Food", "Obese/Overweight").toPandas()
pandas_df6.head()

,High_Caloric_Food,Obese/Overweight
0,no,No
1,no,No
2,no,No
3,no,Yes
4,no,Yes


In [56]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df6['High_Caloric_Food'] = label_encoder.fit_transform(pandas_df6['High_Caloric_Food'])
pandas_df6['Obese/Overweight'] = label_encoder.fit_transform(pandas_df6['Obese/Overweight'])

In [57]:
# Define features (X) and target (y)
X = pandas_df6[['High_Caloric_Food']]
y = pandas_df6['Obese/Overweight']   

In [58]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [59]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [61]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [62]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7312 - loss: 0.6876 - val_accuracy: 0.7544 - val_loss: 0.6516
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.7249 - loss: 0.6451 - val_accuracy: 0.7544 - val_loss: 0.6027
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.7240 - loss: 0.6105 - val_accuracy: 0.7544 - val_loss: 0.5643
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.7382 - loss: 0.5750 - val_accuracy: 0.7544 - val_loss: 0.5387
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.7143 - loss: 0.5906 - val_accuracy: 0.7544 - val_loss: 0.5326
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.7362 - loss: 0.5676 - val_accuracy: 0.7544 - val_loss: 0.5297
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.7248 - loss: 0.5696 - val_accuracy: 0.7544 - val_loss: 0.5279
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.7251 - loss: 0.5771 - val_accura

In [63]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.5323 
Neural Network Accuracy: 0.7352246046066284


## 7. Monitoring calories as predictor of obesity/overweight

In [64]:
# Select columns
data7 = df[['Monitor_Calories', 'Obese/Overweight']]
data7.head()

Row(Monitor_Calories='no', Obese/Overweight='No')

In [65]:
# Convert to pandas dataframe
pandas_df7 = data7.select("Monitor_Calories", "Obese/Overweight").toPandas()
pandas_df7.head()

,Monitor_Calories,Obese/Overweight
0,no,No
1,yes,No
2,no,No
3,no,Yes
4,no,Yes


In [66]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df7['Monitor_Calories'] = label_encoder.fit_transform(pandas_df7['Monitor_Calories'])
pandas_df7['Obese/Overweight'] = label_encoder.fit_transform(pandas_df7['Obese/Overweight'])

In [67]:
# Define features (X) and target (y)
X = pandas_df7[['Monitor_Calories']]
y = pandas_df7['Obese/Overweight']   

In [68]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [69]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [70]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [71]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [72]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7070 - loss: 0.7079 - val_accuracy: 0.7426 - val_loss: 0.6905
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.7470 - loss: 0.6643 - val_accuracy: 0.7426 - val_loss: 0.6471
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - accuracy: 0.7367 - loss: 0.6295 - val_accuracy: 0.7426 - val_loss: 0.6083
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.7444 - loss: 0.5967 - val_accuracy: 0.7426 - val_loss: 0.5797
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.7522 - loss: 0.5710 - val_accuracy: 0.7426 - val_loss: 0.5671
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.7288 - loss: 0.5815 - val_accuracy: 0.7426 - val_loss: 0.5616
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.7275 - loss: 0.5845 - val_accuracy: 0.7426 - val_loss: 0.5603
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.7389 - loss: 0.5709 - val_accura

In [73]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7547 - loss: 0.5536 
Neural Network Accuracy: 0.7494089603424072


## 8. Gender as predictor of obesity

In [80]:
# Select columns
data8 = df[['Gender', 'Obese/Overweight']]
data8.head()

Row(Gender='Female', Obese/Overweight='No')

In [81]:
# Convert to pandas dataframe
pandas_df8 = data8.select("Gender", "Obese/Overweight").toPandas()
pandas_df8.head()

,Gender,Obese/Overweight
0,Female,No
1,Female,No
2,Male,No
3,Male,Yes
4,Male,Yes


In [82]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df8['Gender'] = label_encoder.fit_transform(pandas_df8['Gender'])
pandas_df8['Obese/Overweight'] = label_encoder.fit_transform(pandas_df8['Obese/Overweight'])

In [83]:
# Define features (X) and target (y)
X = pandas_df8[['Gender']]
y = pandas_df8['Obese/Overweight']   

In [84]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [85]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [86]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [87]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [88]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5257 - loss: 0.6911 - val_accuracy: 0.7544 - val_loss: 0.6307
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - accuracy: 0.7343 - loss: 0.6279 - val_accuracy: 0.7544 - val_loss: 0.5907
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.7432 - loss: 0.5936 - val_accuracy: 0.7544 - val_loss: 0.5728
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step - accuracy: 0.7420 - loss: 0.5786 - val_accuracy: 0.7544 - val_loss: 0.5622
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.7406 - loss: 0.5731 - val_accuracy: 0.7544 - val_loss: 0.5594
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - accuracy: 0.7498 - loss: 0.5610 - val_accuracy: 0.7544 - val_loss: 0.5584
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - accuracy: 0.7532 - loss: 0.5560 - val_accuracy: 0.7544 - val_loss: 0.5572
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.7420 - loss: 0.5663 - val_accura

In [89]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.7515 - loss: 0.5651
Neural Network Accuracy: 0.7352246046066284
